I am trying to test reinjection for different structures. this is just a general test.

the idea is to remove the acsl in combined file in a new column, and then reinject right after.

combined_file -> code_implementation -> combined_file

In [40]:
import pandas as pd
import re
import ast
import modal
from validation_treesitter import inject_acsl_specs_treesitter, extract_structure, validate_and_inject

In [41]:
df = pd.read_csv('casp_pt4_1.csv')
df.drop(columns=['description','skeleton_w_acsl','skeleton'], inplace=True)
df.head()

,combined_file,unit_tests,spec
0,/*@ assigns \nothing;\n ensures \result == ...,#include <assert.h>\n\nint main() {\n /* Va...,"[[], '/*@ assigns \\nothing;\n ensures \\re..."
1,/*@ requires n > 0;\n assigns \nothing;\n ...,#include <assert.h>\n\nint main() {\n /* Te...,"[[], '/*@ requires n > 0;\n assigns \\nothi..."
2,#include <limits.h>\n#include <string.h>\n/*@ ...,#include <assert.h>\n\nint main() {\n // Te...,"[[""/*@ predicate matcher_ab(char * x0) = ((((..."
3,/*@\n\trequires n>0;\n\trequires 0<=k<n;\n\tre...,#include <assert.h>\n\nint main() {\n // Te...,"[[], '/*@\n\trequires n>0;\n\trequires 0<=k<n;..."
4,/*@\n\trequires n>0;\n\trequires\valid(a+ (0.....,#include <assert.h>\n\nint main() {\n // Te...,"[[], '/*@\n\trequires n>0;\n\trequires\\valid(..."


In [42]:
def remove_all_comments(c_code: str) -> str:
    if pd.isna(c_code) or not c_code:
        return ""
    c_code = re.sub(r'/\*.*?\*/', '', c_code, flags=re.DOTALL)
    c_code = re.sub(r'//.*?$', '', c_code, flags=re.MULTILINE)
    c_code = re.sub(r'\n\s*\n+', '\n\n', c_code)
    
    return c_code.strip()

df['code_implementation'] = df['combined_file'].apply(remove_all_comments)
df.head()

,combined_file,unit_tests,spec,code_implementation
0,/*@ assigns \nothing;\n ensures \result == ...,#include <assert.h>\n\nint main() {\n /* Va...,"[[], '/*@ assigns \\nothing;\n ensures \\re...","int clamp(int v, int min, int max)\n{\n\tint l..."
1,/*@ requires n > 0;\n assigns \nothing;\n ...,#include <assert.h>\n\nint main() {\n /* Te...,"[[], '/*@ requires n > 0;\n assigns \\nothi...",int pick_index(int n) { return 0; }\n\nint pic...
2,#include <limits.h>\n#include <string.h>\n/*@ ...,#include <assert.h>\n\nint main() {\n // Te...,"[[""/*@ predicate matcher_ab(char * x0) = ((((...",#include <limits.h>\n#include <string.h>\n\nin...
3,/*@\n\trequires n>0;\n\trequires 0<=k<n;\n\tre...,#include <assert.h>\n\nint main() {\n // Te...,"[[], '/*@\n\trequires n>0;\n\trequires 0<=k<n;...","void q2(int a[],int n,int k)\n{\n\n\tfor (int ..."
4,/*@\n\trequires n>0;\n\trequires\valid(a+ (0.....,#include <assert.h>\n\nint main() {\n // Te...,"[[], '/*@\n\trequires n>0;\n\trequires\\valid(...","void bubbleinnerloop(int a[], int n)\n{\n\n\tf..."


In [43]:
row = df.iloc[2]
code_impl = row['code_implementation']

print('Initial:\n' + row['combined_file'] + '\n')
print('Reinjected:\n' + inject_acsl_specs_treesitter(ast.literal_eval(row['spec']), code_impl) + '\n')


Initial:
#include <limits.h>
#include <string.h>
/*@ predicate matcher_ab(char  * x0) = ((((x0[0]=='\0')) ? (\false) : (((x0[0]=='a') &&
\false))) || (((x0[0]=='\0')) ? (\false) : (((x0[0]=='a') &&
((((x0+1)[0]=='\0')) ? (\false) : ((((x0+1)[0]=='b') &&
(((x0+1)+1)[0]=='\0'))))))));*/
/*@
requires ((strlen(x0)>=0) &&
\valid(x0+(0..(strlen(x0)+1)-1)));
assigns \nothing;
ensures \result <==> matcher_ab(x0);
*/
int matcher_ab(char  * x0) {
  char x2 = x0[0];
  int x3 = x2 == '\0';
  int x7;
  if (x3) {
    x7 = 0;
  } else {
    int x4 = x2 == 'a';
    int x6;
    if (x4) {
      x6 = 0;
    } else {
      x6 = 0;
    }
    x7 = x6;
  }
  int x17;
  if (x3) {
    x17 = 0;
  } else {
    int x4 = x2 == 'a';
    int x16;
    if (x4) {
      char  *x5 = x0+1;
      char x8 = x5[0];
      int x9 = x8 == '\0';
      int x15;
      if (x9) {
        x15 = 0;
      } else {
        int x10 = x8 == 'b';
        int x14;
        if (x10) {
          char  *x11 = x5+1;
          char x12 = x11[0];


In [44]:
def reinject_row(row):
    try:
        spec = ast.literal_eval(row['spec'])
        return inject_acsl_specs_treesitter(spec, row['code_implementation'])
    except Exception as e:
        return f"ERROR: {e}"

df['reinjection'] = df.apply(reinject_row, axis=1)
df.head()

,combined_file,unit_tests,spec,code_implementation,reinjection
0,/*@ assigns \nothing;\n ensures \result == ...,#include <assert.h>\n\nint main() {\n /* Va...,"[[], '/*@ assigns \\nothing;\n ensures \\re...","int clamp(int v, int min, int max)\n{\n\tint l...",/*@ assigns \nothing;\n ensures \result == ...
1,/*@ requires n > 0;\n assigns \nothing;\n ...,#include <assert.h>\n\nint main() {\n /* Te...,"[[], '/*@ requires n > 0;\n assigns \\nothi...",int pick_index(int n) { return 0; }\n\nint pic...,/*@ requires n > 0;\n assigns \nothing;\n ...
2,#include <limits.h>\n#include <string.h>\n/*@ ...,#include <assert.h>\n\nint main() {\n // Te...,"[[""/*@ predicate matcher_ab(char * x0) = ((((...",#include <limits.h>\n#include <string.h>\n\nin...,/*@ predicate matcher_ab(char * x0) = ((((x0[...
3,/*@\n\trequires n>0;\n\trequires 0<=k<n;\n\tre...,#include <assert.h>\n\nint main() {\n // Te...,"[[], '/*@\n\trequires n>0;\n\trequires 0<=k<n;...","void q2(int a[],int n,int k)\n{\n\n\tfor (int ...",/*@\n\trequires n>0;\n\trequires 0<=k<n;\n\tre...
4,/*@\n\trequires n>0;\n\trequires\valid(a+ (0.....,#include <assert.h>\n\nint main() {\n // Te...,"[[], '/*@\n\trequires n>0;\n\trequires\\valid(...","void bubbleinnerloop(int a[], int n)\n{\n\n\tf...",/*@\n\trequires n>0;\n\trequires\valid(a+ (0.....


In [45]:
df.drop(columns=['code_implementation','spec'], inplace=True)

In [46]:
df.head()

,combined_file,unit_tests,reinjection
0,/*@ assigns \nothing;\n ensures \result == ...,#include <assert.h>\n\nint main() {\n /* Va...,/*@ assigns \nothing;\n ensures \result == ...
1,/*@ requires n > 0;\n assigns \nothing;\n ...,#include <assert.h>\n\nint main() {\n /* Te...,/*@ requires n > 0;\n assigns \nothing;\n ...
2,#include <limits.h>\n#include <string.h>\n/*@ ...,#include <assert.h>\n\nint main() {\n // Te...,/*@ predicate matcher_ab(char * x0) = ((((x0[...
3,/*@\n\trequires n>0;\n\trequires 0<=k<n;\n\tre...,#include <assert.h>\n\nint main() {\n // Te...,/*@\n\trequires n>0;\n\trequires 0<=k<n;\n\tre...
4,/*@\n\trequires n>0;\n\trequires\valid(a+ (0.....,#include <assert.h>\n\nint main() {\n // Te...,/*@\n\trequires n>0;\n\trequires\valid(a+ (0.....


In [35]:
df.to_csv('compare.csv', index=False)

now want to see if the reinjetion passes

In [39]:
# Get reference to the deployed Modal function
check_compilation = modal.Function.from_name("comp_check", "check_compilation")

# Spawn all jobs in parallel
reinjection_codes = df['reinjection'].tolist()
handles = [check_compilation.spawn(code) for code in reinjection_codes]
print(f"All jobs spawned. Collecting results...")

# Collect results
results = [h.get() for h in handles]

# Add results as new column
df['compiles'] = results

# Summary
print(f"\nTotal: {len(df)}")
print(f"Compiles: {sum(results)}")
print(f"Fails: {len(results) - sum(results)}")
print(f"Pass rate: {sum(results)/len(results)*100:.1f}%")

# Show failures
failures = df[~df['compiles']]
print(f"\nFailed rows: {len(failures)}")
if len(failures) > 0:
    print(failures.index.tolist()[:20])


All jobs spawned. Collecting results...

Total: 354
Compiles: 354
Fails: 0
Pass rate: 100.0%

Failed rows: 0


In [47]:
# Combine reinjection with unit tests for runtime testing
df['reinjection_with_tests'] = df['reinjection'] + '\n\n' + df['unit_tests']

# Get reference to the deployed runtime check function
check_runtime = modal.Function.from_name("run_check", "check_runtime")

# Spawn all runtime checks in parallel
test_codes = df['reinjection_with_tests'].tolist()
print(f"Spawning {len(test_codes)} runtime checks...")

handles = [check_runtime.spawn(code) for code in test_codes]
print(f"All jobs spawned. Collecting results...")

# Collect results
runtime_results = [h.get() for h in handles]

# Add results as new column
df['tests_pass'] = runtime_results

# Summary
print(f"\nTotal: {len(df)}")
print(f"Tests pass: {sum(runtime_results)}")
print(f"Tests fail: {len(runtime_results) - sum(runtime_results)}")
print(f"Pass rate: {sum(runtime_results)/len(runtime_results)*100:.1f}%")

# Show failures
test_failures = df[~df['tests_pass']]
print(f"\nFailed rows: {len(test_failures)}")
if len(test_failures) > 0:
    print(test_failures.index.tolist()[:20])


Spawning 354 runtime checks...
All jobs spawned. Collecting results...

Total: 354
Tests pass: 354
Tests fail: 0
Pass rate: 100.0%

Failed rows: 0


In [51]:
# Test reinjection with Frama-C verification
verify_annotated_c = modal.Function.from_name("frama_check", "verify_annotated_c")

# Spawn all Frama-C verification jobs in parallel
reinjection_codes = df['combined_file'].tolist()
print(f"Spawning {len(reinjection_codes)} Frama-C verification checks...")

handles = [verify_annotated_c.spawn(code) for code in reinjection_codes]
print(f"All jobs spawned. Collecting results...")

# Collect results
frama_results = [h.get() for h in handles]

# Add results as new column
df['frama_verified'] = frama_results

# Summary
print(f"\nTotal: {len(df)}")
print(f"Verified: {sum(frama_results)}")
print(f"Not verified: {len(frama_results) - sum(frama_results)}")
print(f"Pass rate: {sum(frama_results)/len(frama_results)*100:.1f}%")

# Show failures
frama_failures = df[~df['frama_verified']]
print(f"\nFailed rows: {len(frama_failures)}")
if len(frama_failures) > 0:
    print(frama_failures.index.tolist()[:20])


Spawning 354 Frama-C verification checks...
All jobs spawned. Collecting results...

Total: 354
Verified: 347
Not verified: 7
Pass rate: 98.0%

Failed rows: 7
[19, 29, 36, 42, 86, 96, 285]


In [52]:
# Re-run Frama-C on failed rows only
failed_indices = [19, 29, 36, 42, 86, 96, 285]

verify_annotated_c = modal.Function.from_name("frama_check", "verify_annotated_c")

# Get codes for failed rows
failed_codes = [df.iloc[idx]['combined_file'] for idx in failed_indices]
print(f"Re-running {len(failed_codes)} failed rows...")

handles = [verify_annotated_c.spawn(code) for code in failed_codes]
print(f"All jobs spawned. Collecting results...")

retry_results = [h.get() for h in handles]

# Show results
for idx, result in zip(failed_indices, retry_results):
    status = "✓ PASS" if result else "✗ FAIL"
    print(f"Row {idx}: {status}")

# Update dataframe with new results
for idx, result in zip(failed_indices, retry_results):
    df.at[idx, 'frama_verified'] = result

# Summary
still_failing = [idx for idx, result in zip(failed_indices, retry_results) if not result]
print(f"\nStill failing: {len(still_failing)} rows")
if still_failing:
    print(still_failing)


Re-running 7 failed rows...
All jobs spawned. Collecting results...
Row 19: ✗ FAIL
Row 29: ✗ FAIL
Row 36: ✗ FAIL
Row 42: ✗ FAIL
Row 86: ✗ FAIL
Row 96: ✗ FAIL
Row 285: ✗ FAIL

Still failing: 7 rows
[19, 29, 36, 42, 86, 96, 285]
